In [6]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import uproot  # Pour lire les fichiers ROOT sans PyROOT
import pandas as pd
import awkward as ak  # uproot utilise awkward pour gérer les structures complexes
import pickle
import os

In [30]:
def process_root_file(filename, histname_incident, histname_IP, rebin):
    try:
        # Ouvrir le fichier ROOT
        file = uproot.open(filename)

        # Initialiser un dictionnaire pour stocker les résultats
        results = {}

        # Lire l'histogramme
        if histname_IP not in file:
            print(f"Erreur : L'histogramme {histname_IP} n'existe pas dans le fichier.")
            return results

        hist1 = file[histname_IP]
        bin_values = hist1.values()
        hist_inc = file[histname_incident]
        bin_values_inc = hist_inc.values()
        integral=0

        # Appliquer le rebinning
        if rebin > 1:
            bin_values_inc = np.sum(bin_values_inc.reshape(-1, rebin), axis=1)

        # Ajouter les valeurs des bins à la suite des résultats
        for i, value in enumerate(bin_values):
            integral += value
            #print(f"integral = {integral}")

        print(f"integral = {integral}")
        integral_bis = integral/(3.1415*6.5*6.5)
        print(f"integral = {integral_bis}")

        # Ajouter les valeurs des bins à la suite des résultats
        for i, value in enumerate(bin_values_inc):
            results[f'Bin_{i+1}'] = value/integral_bis
        
        # Ajouter les valeurs des bins à la suite des résultats
        for i, value in enumerate(bin_values):
            results[f'PSL_IP_{i+1}'] = value/integral
            

        return results
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [31]:
def serialize_data(data, output_file):
    """
    Serialize data to a pickle file.

    Parameters:
    data (dict): The dictionary to serialize.
    output_file (str): The path to the output pickle file.
    """
    try:
        with open(output_file, 'wb') as fichier:
            pickle.dump(data, fichier)
        print(f"Les données ont été sérialisées et écrites dans {output_file}")
    except Exception as e:
        print(f"An error occurred during serialization: {e}")

In [32]:
def deserialize_data(output_file):
    """
    Deserialize data from a pickle file.

    Parameters:
    output_file (str): The path to the input pickle file.

    Returns:
    pd.DataFrame: The deserialized DataFrame.
    """
    try:
        with open(output_file, 'rb') as fichier:
            data = pickle.load(fichier)
        return data
    except Exception as e:
        print(f"An error occurred during deserialization: {e}")
        return None

In [33]:
def Process(directory, histname_incident, rebin, histname_IP, output_file):
    rebin = int(rebin)
    # Initialiser combined_results en dehors de la condition pour s'assurer qu'il est toujours défini
    if os.path.exists(output_file):
        combined_results = deserialize_data(output_file)
        if combined_results is None:
            # Initialiser avec un DataFrame vide si la désérialisation échoue
            combined_results = pd.DataFrame(columns=[f'Bin_{i+1}' for i in range(int(500/rebin))] + [f'PSL_IP_{i+1}' for i in range(15)])
    else:
        # Initialiser avec un DataFrame vide si le fichier n'existe pas
        combined_results = pd.DataFrame(columns=[f'Bin_{i+1}' for i in range(int(500/rebin))] + [f'PSL_IP_{i+1}' for i in range(15)])

    # Lister tous les fichiers dans le dossier
    for filename in os.listdir(directory):
        if filename.endswith('.root'):
            filepath = os.path.join(directory, filename)
            print(f"Processing file: {filepath}")
            results = process_root_file(filepath, histname_incident, histname_IP, rebin)
            if results is not None:
                # Ajouter une nouvelle ligne au DataFrame avec les résultats
                combined_results = pd.concat([combined_results, pd.DataFrame([results])], ignore_index=True)

    # Afficher toutes les données
    print(combined_results)

    # Sérialiser les données combinées
    serialize_data(combined_results, output_file)

In [34]:
# Exemple d'utilisation
directory = r"D:\PYTHON\Data_ML"
#treename = "IP"
histname_incident = "incident_energy"  # Remplacez par le nom de votre histogramme
histname_IP = "PSL vs IP"  # Remplacez par le nom de votre histogramme
rebin = 1
# Définir le chemin complet du fichier de sortie
output_file = r"D:\PYTHON\Data_ML\test.pickle"

# Check if the file exists and remove it
if os.path.exists(output_file):
    os.remove(output_file)
    print(f"Le fichier existant {output_file} a été supprimé.")

Process(directory, histname_incident, rebin, histname_IP, output_file)

Le fichier existant D:\PYTHON\Data_ML\test.pickle a été supprimé.
Processing file: D:\PYTHON\Data_ML\Exp_100keV_100M.root
integral = 46529.250870072065
integral = 350.55993769284123
Processing file: D:\PYTHON\Data_ML\Exp_101keV_100M.root
integral = 46411.50592351801
integral = 349.6728255997861
Processing file: D:\PYTHON\Data_ML\Exp_102keV_100M.root
integral = 46365.78974004113
integral = 349.32839146144244
Processing file: D:\PYTHON\Data_ML\Exp_103keV_100M.root
integral = 46257.5744554742
integral = 348.51307759530846
Processing file: D:\PYTHON\Data_ML\Exp_104keV_100M.root
integral = 46154.78710990323
integral = 347.7386588203406
Processing file: D:\PYTHON\Data_ML\Exp_105keV_100M.root
integral = 46111.182496842164
integral = 347.41013364205025
Processing file: D:\PYTHON\Data_ML\Exp_106keV_100M.root
integral = 46012.18713674948
integral = 346.664284383422
Processing file: D:\PYTHON\Data_ML\Exp_107keV_100M.root
integral = 45962.26120028884
integral = 346.28813319148105
Processing file: 

C:\Users\huber\AppData\Local\Temp\ipykernel_3720\1753591131.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_results = pd.concat([combined_results, pd.DataFrame([results])], ignore_index=True)


integral = 44213.78549413189
integral = 333.11479549216125
Processing file: D:\PYTHON\Data_ML\Exp_131keV_100M.root
integral = 44090.31981536469
integral = 332.18458234996615
Processing file: D:\PYTHON\Data_ML\Exp_132keV_100M.root
integral = 43997.08829462193
integral = 331.4821589175783
Processing file: D:\PYTHON\Data_ML\Exp_133keV_100M.root
integral = 43995.336255672286
integral = 331.46895873374694
Processing file: D:\PYTHON\Data_ML\Exp_134keV_100M.root
integral = 43867.321072926294
integral = 330.50446879144187
Processing file: D:\PYTHON\Data_ML\Exp_135keV_100M.root
integral = 43904.719049679545
integral = 330.786232029734
Processing file: D:\PYTHON\Data_ML\Exp_136keV_100M.root
integral = 43765.81860293896
integral = 329.73973050554537
Processing file: D:\PYTHON\Data_ML\Exp_137keV_100M.root
integral = 43590.30063945624
integral = 328.4173458723972
Processing file: D:\PYTHON\Data_ML\Exp_138keV_100M.root
integral = 43563.66571537353
integral = 328.21667345338574
Processing file: D:\PY